In [ ]:
import sys

import numpy as np
import matplotlib.pyplot as plt

import tqdm

# CSAF Imports
import csaf.config as cconf
import csaf.system as csys
import run_parallel as rp  # for now, batch simulations is not in CSAF package



In [ ]:
import csaf.config as cconf
import csaf.system as csys

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_multiagent_config.toml")

In [ ]:
class AcasSampler:
    @staticmethod
    def air_collision_condition(ctraces):
        """ground collision premature termnation condition
        TODO: I had to alter the terminating conditions to support this expression
        """
        # get the aircraft states
        sa, sb = ctraces['planta']['states'], ctraces['plantb']['states']
        if sa and sb:
            # look at distance between last state
            return (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sb[-1][9:11]))) < 10
        
    def __init__(self, conf, tspan=(0.0, 20.0)):
        self.config = conf
        self.tspan = tspan
        
        
    def run_samples(self, x_samp):
            """given an array of plant initial states, 
            run batches of simulations to see if they meet the termination criteria"""
            # assign samples to the system plant
            init_states = [{"planta": xai, "plantb": xbi} for xai, xbi in x_samp]
            res = rp.run_workgroup(len(init_states), self.config, init_states, self.tspan,
                                   terminating_conditions_all=self.air_collision_condition)
            return res
        
    def fit(self, X, y):
        """this model cannot be trained"""
        pass
    
    def predict(self, X):
        """classification prediction"""
        ret = self.run_samples(X)
        return np.array([b for b,_, _ in ret])

In [ ]:
# set the scenario states
x_samp = []
for _ in range(32):
    xa = [500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6000.0, 9.0]
    xe, xn = np.random.uniform(200, 5000), np.random.uniform(200, 5000)
    heading = np.random.uniform(-np.pi, np.pi)
    xb = [500.0, 0.0, 0.0, 0.0, 0.0, heading, 0.0, 0.0, 0.0, xn, xe, 6000.0, 9.0]
    x_samp.append((xa, xb))

In [ ]:
sampler = AcasSampler(my_conf)
sim_res = sampler.run_samples(x_samp)

In [ ]:
[passed for passed, _, _ in sim_res]